## 数据处理流程
1. 数据读入
2. 数据清洗
3. 数据无量纲化处理
4. 特征选择
5. 特征提取
6. 聚类

# 将数据文件处理成电脑可处理的小规模的数据

## 数据读入

In [1]:
# import libs
import numpy as np
import pandas as pd
import os
from sklearn.cluster import KMeans

In [2]:
# 从文件夹'newdata/'中读取数据文件
filedir = '/mnt/hgfs/vmfiles/newdata/'
files = os.listdir(filedir)

In [3]:
nrows = 10000 # 每个文件中读取前nrows行数据
alldata = pd.DataFrame() #新建空的DataFrame
for f in files: # 循环读入数据，加入到新建的DataFrame中
    tdata = pd.read_csv(filedir + f, nrows=nrows)
    print("get " + str(tdata.shape[0]) + " rows data from " + f)
    alldata = alldata.append(tdata, ignore_index=True)
    #alldata = alldata.append(tdata)

get 10000 rows data from BENIGN_data.csv
get 10000 rows data from DrDoS_DNS_data.csv
get 10000 rows data from DrDoS_LDAP_data.csv
get 10000 rows data from DrDoS_MSSQL_data.csv


/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (86) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


get 10000 rows data from DrDoS_NetBIOS_data.csv
get 10000 rows data from DrDoS_NTP_data.csv
get 10000 rows data from DrDoS_SNMP_data.csv
get 10000 rows data from DrDoS_SSDP_data.csv
get 10000 rows data from DrDoS_UDP_data.csv
get 10000 rows data from Syn_data.csv
get 10000 rows data from TFTP_data.csv


In [4]:
cols = alldata.columns
alldata = alldata.drop(cols[0:2],axis = 1) # 删除两列必然没用的数据
alldata = alldata.dropna(how='any') # 删除含有nan值的数据
alldata.reset_index(inplace=True, drop=True) # 重置index
alldata
# 36079 rows × 87 columns

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,192.168.50.8-125.56.201.115-59099-80-6,192.168.50.8,59099,125.56.201.115,80,6,2018-12-01 10:51:40.379825,110861755,26,26,...,64223.338986,392446.0,179351.0,9.863210e+06,413171.813769,10007500.0,8632089.0,detectportal.firefox.com/success.txt,0,BENIGN
1,192.168.50.8-54.218.239.186-59102-443-6,192.168.50.8,59102,54.218.239.186,443,6,2018-12-01 10:51:40.504696,40335006,9,10,...,43.554563,90287.0,90185.0,9.993447e+06,40495.753715,10018634.0,9933709.0,0,0,BENIGN
2,192.168.50.253-224.0.0.5-0-0-0,192.168.50.253,0,224.0.0.5,0,0,2018-12-01 10:51:41.309691,113244633,56,0,...,859690.585416,2978061.0,4.0,9.188876e+06,809901.667647,9882838.0,6781893.0,0,0,BENIGN
3,192.168.50.8-23.15.4.11-59155-80-6,192.168.50.8,59155,23.15.4.11,80,6,2018-12-01 10:51:43.125459,95628949,21,20,...,177.422362,15367.0,14798.0,1.001418e+07,5184.077926,10016037.0,10000366.0,0,0,BENIGN
4,172.217.0.110-192.168.50.8-80-59131-6,192.168.50.8,59131,172.217.0.110,80,6,2018-12-01 10:51:43.141041,95613243,21,20,...,17.902514,26534.0,26473.0,1.000431e+07,21.430119,10004365.0,10004289.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107839,172.16.0.5-192.168.50.1-49497-64965-6,172.16.0.5,49497,192.168.50.1,64965,6,2018-12-01 13:34:28.374676,1,2,0,...,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,0,1,TFTP
107840,172.16.0.5-192.168.50.1-64068-64068-6,172.16.0.5,64068,192.168.50.1,64068,6,2018-12-01 13:34:28.374742,14075036,4,0,...,0.000000,1.0,1.0,1.407503e+07,0.000000,14075034.0,14075034.0,0,1,TFTP
107841,172.16.0.5-192.168.50.1-47148-47148-6,172.16.0.5,47148,192.168.50.1,47148,6,2018-12-01 13:34:28.374744,48,2,0,...,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,0,1,TFTP
107842,172.16.0.5-192.168.50.1-49498-55027-6,172.16.0.5,49498,192.168.50.1,55027,6,2018-12-01 13:34:28.374793,116,2,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,0,1,TFTP


In [5]:
L = set(alldata[' Label'])
labels = []
for l in L:
    labels.append(l)
labels
    

['BENIGN',
 'DrDoS_DNS',
 'Syn',
 'DrDoS_LDAP',
 'DrDoS_SNMP',
 'DrDoS_NetBIOS',
 'DrDoS_UDP',
 'DrDoS_NTP',
 'DrDoS_SSDP',
 'TFTP',
 'DrDoS_MSSQL']

## 数据清洗

In [6]:
alldata.dropna(how='any',inplace=True) # 再次删除含有空值的行

# 删除非数值型数据，这些数据暂时不用于聚类处理，但不代表这些数据没用
dropset = ['Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
           ' Fwd Header Length.1', 'SimillarHTTP', ' Inbound']
# 删除含有大量负数的列
drop_nega = [' Fwd Header Length', 'Init_Win_bytes_forward',
       ' Init_Win_bytes_backward', ' act_data_pkt_fwd',
       ' min_seg_size_forward']
alldata.drop(dropset, axis=1,inplace=True)
alldata.drop(drop_nega, axis=1,inplace=True)
cols = alldata.columns
# len(dropset),cols.size

# 将labels替代成数值型值
for i in range(len(labels)):
    alldata.replace(to_replace =labels[i], value = i, inplace = True)

In [7]:
# 删除数据中inf的值
idx_tuple = np.where(alldata.max(axis=1).values == np.inf)
idx = list(idx_tuple[0])
alldata.drop(axis=0, index = idx, inplace = True)
alldata.reset_index(inplace=True, drop=True)
# 删除数据中含负数的数据条数
idx_tuple = np.where(alldata.min(axis=1).values < 0)
idx = list(idx_tuple[0])
alldata.drop(axis=0, index = idx, inplace = True)
alldata.reset_index(inplace=True, drop=True)

In [8]:
# 拆分数据X-Y
Y = alldata[' Label']
X = alldata.drop([' Label'], axis=1)

In [9]:
# 将pandas数据转成numpy array
X = X.values
Y = Y.values

## 数据无量纲化处理

In [10]:
from sklearn import preprocessing

In [11]:
# 采用两种不同的标准化方式
#MaxAbs标准化
#建立MinMaxScaler对象
maxabs = preprocessing.MaxAbsScaler()
# 标准化处理
data_maxabs = maxabs.fit_transform(X)

#zscore标准化
zscore = preprocessing.StandardScaler()
#zscore标准化
X_zscore = zscore.fit_transform(X)

## 特征选择

In [12]:
from scipy.stats import pearsonr
from sklearn.feature_selection import SelectKBest
from numpy import array

In [2]:
## 递归特征消除法
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
X_new = RFE(estimator=LogisticRegression(), n_features_to_select=10).fit_transform(data_maxabs, Y)

NameError: name 'data_maxabs' is not defined

In [1]:
X_new.shape

NameError: name 'X_new' is not defined

In [16]:
X_new

array([[0.00000000e+00, 7.85738800e-03, 3.62178850e-02, ...,
        1.00000000e+00, 6.61601596e-03, 7.85738800e-03],
       [0.00000000e+00, 2.94793778e-04, 1.50044980e-04, ...,
        1.00000000e+00, 9.28228422e-04, 2.94793778e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [17]:
data_maxabs

array([[9.23848282e-01, 3.02698675e-04, 5.64971751e-03, ...,
        6.25990229e-03, 8.72147317e-02, 7.52281115e-02],
       [3.36125168e-01, 1.04780311e-04, 2.17296827e-03, ...,
        6.13544905e-04, 8.73117638e-02, 8.65716478e-02],
       [9.43705605e-01, 6.51966377e-04, 0.00000000e+00, ...,
        1.22706950e-02, 8.61283102e-02, 5.91037700e-02],
       ...,
       [4.00000140e-07, 2.32845135e-05, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.66667005e-07, 2.32845135e-05, 4.34593655e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.33333625e-09, 2.32845135e-05, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [18]:
# X_new = SelectKBest(lambda X, Y: array(list(map(lambda x:pearsonr(x, Y), X.o))).T, k=30).fit_transform(data_maxabs, Y)

## 将矩阵转换成matlab处理的.mat格式

In [22]:
import scipy.io as io
xmatpath = '/mnt/hgfs/vmfiles/mat/X.mat'
ymatpath = '/mnt/hgfs/vmfiles/mat/Y.mat'
xmat = X_new
ymat = Y
io.savemat(xmatpath,{'X':xmat})
io.savemat(ymatpath,{'Y':ymat})

In [23]:
Y

array([0, 0, 0, ..., 9, 9, 9])

In [24]:
set(Y)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}